Nasledujúci skript nám umožní vyfiltrovať textové zmluvy. Nad tými zmluvami, ktoré boli naskenované vieme zbehnúť OCR - strojový prevod naskenovaných dokumentov do textu.

In [ ]:
import os
import re

if not os.path.exists('contracts_text'):
    os.makedirs('contracts_text')

if not os.path.exists('contracts_scan'):
    os.makedirs('contracts_scan')

raw_dir   = os.getcwd()+'\\contracts\\'
text_dir = os.getcwd()+'\\contracts_text\\'
scan_dir = os.getcwd()+'\\contracts_scan\\'

contracts = [f for f in os.listdir(raw_dir) if os.path.isfile(os.path.join(raw_dir, f))]

i = 0
n = len(contracts)

for contract in contracts:
    i += 1
    print('Processing contract ',contract,' ',i,' out of ',n)

    try:
        os.system('pdftotext '+raw_dir+contract+' output.txt')
    
        fo = open('output.txt','r', encoding='utf8')
        lines = fo.readlines()
        fo.close()

        file = ''
        for line in lines:
            file += line

        del lines
        
        check_file   = re.sub(r'\s+', '', file, flags=re.UNICODE)

        if (len(check_file)>0):
            os.system('move '+raw_dir+contract+' '+text_dir+contract)
            os.system('move output.txt '+text_dir+contract.replace('.pdf','.txt'))
            print('Moved to '+text_dir)
        else:
            os.system('move '+raw_dir+contract+' '+scan_dir+contract)
            print('Moved to '+scan_dir)

    except:
        os.system('move '+raw_dir+contract+' '+scan_dir+contract)
        print('Moved to '+scan_dir)

Dokumenty, ktoré boli naskenované sa môžeme pokúsiť previesť na text pomocou programu Tesseract od Googlu.

In [ ]:
import os
import subprocess
import re

find_number = re.compile(r'\d+')
FNULL = open(os.devnull, 'w')

def natural_sort(l):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

scan_dir = os.getcwd()+'\\contracts_scan\\'
contracts = [f for f in os.listdir(scan_dir) if os.path.isfile(os.path.join(scan_dir, f))]

N_contracts = len(contracts)

for i,contract in enumerate(contracts):
    print('Processing contract:',contract,' ',i+1,'out of',N_contracts)

    # Convert PDF to PNG images
    # -r specifies DPI
    # -gray is for grayscale image
    print('\tRendering pages from PDF ...')

    retcode = subprocess.call('pdftopng -q -r 350 -gray '+os.path.join(scan_dir,contract)+' '+os.path.join(scan_dir,'output'), stdout=FNULL, stderr=subprocess.STDOUT)
    output_files = [f for f in os.listdir(scan_dir) if os.path.isfile(os.path.join(scan_dir, f)) and 'output' in f]
    output_files = natural_sort(output_files)

    # Detect rotation and auto-rotate image
    # ...

    # Perform OCR by Tesseract
    text = ''
    for page in output_files:
        print('\tOCR on page:',int(find_number.findall(page)[0]))
        retcode = subprocess.call('tesseract '+os.path.join(scan_dir,page)+' '+os.path.join(scan_dir,'output')+' -l eng+ces+slk', stdout=FNULL, stderr=subprocess.STDOUT)
        fo = open(os.path.join(scan_dir,'output.txt'),'r', encoding = 'utf-8')
        lines = fo.readlines()
        fo.close()

        for line in lines:
            text += line

    # Save results
    fo = open(os.path.join(scan_dir,contract.replace('.pdf','.txt')),'w', encoding = 'utf-8')
    fo.write(text)
    fo.close()

    # Delete temporary files
    os.system('del '+os.path.join(scan_dir,'output*'))

Po vykonaní OCR môžeme, dokumenty ku ktorým bla úspešne vykonaná OCR analýza, previesť naspäť do priečinku z textovými dokumentami.

In [ ]:
import os
import re

text_dir = os.getcwd()+'\\contracts_text\\'
scan_dir = os.getcwd()+'\\contracts_scan\\'

contracts_pdf = [f for f in os.listdir(scan_dir) if os.path.isfile(os.path.join(scan_dir, f)) and 'pdf' in f]
contracts_txt = [f for f in os.listdir(scan_dir) if os.path.isfile(os.path.join(scan_dir, f)) and 'txt' in f]

for contract in contracts_txt:
    os.system('move '+scan_dir+contract+' '+text_dir+contract)
    os.system('move '+scan_dir+contract.replace('txt','pdf')+' '+text_dir+contract.replace('txt','pdf'))